In [1]:
import numpy as np
import pandas as pd
import os
import folium
from folium import plugins
import pygeohash as geo
from datetime import datetime
import calendar
from datetime import timezone
import datetime
import pytz
import geopy.distance
from shapely.geometry import Point
import geopandas as gpd
import glob
from sklearn.base import BaseEstimator, TransformerMixin

In [2]:
def retrieve_tables(census_block_groups:[], features:{}, path='Census_Data/data_tables'):
    all_files = glob.glob(path+"/*.csv")
    combined = pd.DataFrame({'census_block_group': census_block_groups})
    combined.census_block_group = combined.census_block_group.astype(str)
    for file in all_files:
        df = pd.read_csv(file)
        df = df.loc[df.census_block_group.isin(census_block_groups)].reset_index(drop=True)
        df.census_block_group = df.census_block_group.astype(str)
        combined = combined.merge(df, on='census_block_group')
    combined = combined[(['census_block_group']+[i for i in features])]
    combined.rename(features, axis=1, inplace=True)
    return combined    

In [3]:
def get_census_block_groups(df, location = None, lat_col = 'home_lat', long_col = 'home_long', drop =True):
    geom = gpd.read_file('Census_Data/cbg.geojson')
    if location != None:
        df = df[df.batch_id.str.contains(location)]
    df = gpd.GeoDataFrame(df, geometry = gpd.points_from_xy(df[long_col], df[lat_col]))
    df = gpd.sjoin(df, geom, op='within')
    if drop:
        df = df.drop(['geometry','index_right','StateFIPS','CountyFIPS','TractCode','BlockGroup','MTFCC'], axis=1)
    return df

In [4]:
census_dic = {  'B01001e1': 'population_total',
                'B23025e3': 'civilian_labor_force_total', 
                'B23025e5': 'civilian_labor_force_unemployed',
                'B09002e1': 'have_child_under18',            
                'B25074e1': 'renter_occupied_units',
                'B25049e1': 'total_occupied_units',
                'B25075e1': 'owner_occupied_units',
                'B27010e18': 'age_19to34',
                'B27010e2': 'age_under19',
                'B27010e34': 'age_35to64',
                'B27010e51': 'age_65&over',
                'B27010e17': 'under19_NoHealth',
                'B27010e33': '19to34_NoHealth',
                'B27010e50': '35to64_NoHealth',
                'C21007e11': 'nonvet_povertylvl_18to64',
                'C21007e4': 'vet_povertylvl_18to64',
                'C21007e26': 'nonvet_povertylvl_65&over',
                'C21007e19': 'vet_povertylvl_65&over',
                'C21007e12': 'nonvet_povertylvl_disability_18to64',
                'C21007e15': 'nonvet_abovepovertylvl_disability_18to64',
                'C21007e5': 'vet_povertylvl_disability_18to64',
                'C21007e8': 'vet_abovepovertylvl_disability_18to64',
                'C21007e27': 'nonvet_povertylvl_disability_65&over',
                'C21007e30': 'nonvet_abovepovertylvl_disability_65&over',
                'C21007e20': 'vet_povertylvl_disability_65&over',
                'C21007e23': 'vet_abovepovertylvl_disability_65&over',
                'B25014e5': 'owner_occupied_1.01to1.5_PeoplePerRoom',
                'B25014e6': 'owner_occupied_1.51to2_PeoplePerRoom',
                'B25014e7': 'owner_occupied_2.01orMore_PeoplePerRoom',
                'B25014e11': 'renter_occupied_1.01to1.5_PeoplePerRoom',
                'B25014e12': 'renter_occupied_1.5to2_PeoplePerRoom',
                'B25014e13': 'renter_occupied_2.01orMore_PeoplePerRoom',
                'B19013e1': 'Median_Household_Income',
                'B02001e2': 'White alone',
                'B02001e3': 'Black or African American alone',
                'B02001e4': 'American Indian and Alaska Native alone',
                'B02001e5': 'Asian alone',
                'B02001e6': 'Native Hawaiian and Other Pacific Islander alone',
                'B02001e7': 'Some other race alone',
                'B02001e8': 'Two or more races',
                'B01001e2': 'Males',
                'B01001e26':'Females',
                'C21007e2': 'age18to64',
                'C21007e17': 'age65&over',
                'B15003e1': 'age25&over'
               }

#B23027e35 full time estimates
#B23022e10 hours per week
#above number of workers in household and children

In [5]:
fields = pd.read_csv('Census_Data/cbg_field_descriptions.csv')

#age_sex_dic = {fields.loc[i,'table_id']:str(fields.loc[i,'field_level_5'])+str(fields.loc[i,'field_level_6']) for i in range(len(fields)) if fields.loc[i,'table_title']=='Sex By Age' and fields.loc[i,'field_level_1']=='Estimate'}
#race_dic = {fields.loc[i,'table_id']:str(fields.loc[i,'field_level_5']) for i in range(len(fields)) if fields.loc[i,'table_title']=='Race' and fields.loc[i,'field_level_1']=='Estimate'}
education_dic = {fields.loc[i,'table_id']:str(fields.loc[i,'field_level_5']) for i in range(len(fields)) if fields.loc[i,'table_title']=='Educational Attainment For The Population 25 Years And Over' and fields.loc[i,'field_level_1']=='Estimate'}

In [6]:
del education_dic['B15003e1']
new_census_dic = {**census_dic, **education_dic}

In [7]:
#user_list = pd.read_excel('DCBalt_user_list.xlsx')
user_list = pd.read_excel('Final_integrated_files_fixed/DCBalt_user_list.xlsx')
new_user_list = get_census_block_groups(user_list, location = 'DC')
census_table = retrieve_tables(list(new_user_list.CensusBlockGroup.unique()), new_census_dic)

/apps/software/standard/compiler/gcc/9.2.0/jupyter_conda/2020.11-py3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3418: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
<ipython-input-3-f9d6db566f86>:6: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  df = gpd.sjoin(df, geom, op='within')


In [8]:
#notes: think about changing people rooms per person

class TransformationsPipeline(BaseEstimator, TransformerMixin):
    def __init__(self, matrix = False):
        self.matrix = False
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X['civilian_laborforce_unemployment'] = X.civilian_labor_force_unemployed/X.civilian_labor_force_total
        X = X.drop(['civilian_labor_force_unemployed','civilian_labor_force_total'], axis=1)
        
        # proportion renters out of occupied
        X['pct_renter_occupied'] = X.renter_occupied_units/X.total_occupied_units
        
        #non-institutionalized
        X['pct_19to35'] = X.age_19to34/X.population_total
        X['pct_35to64'] = X.age_35to64/X.population_total
        X['pct_65over'] = X['age_65&over']/X.population_total
        X = X.drop(['age_35to64','age_19to34','age_65&over','age_under19'], axis=1)
        
        X['no_health_under65'] = (X.under19_NoHealth + X['19to34_NoHealth'] + X['35to64_NoHealth'])/X.population_total
        X = X.drop(['under19_NoHealth','19to34_NoHealth','35to64_NoHealth'], axis=1)
        
        #civilian adults (does not specify non-instituionalized)
        X['civilian_adults_total'] = X['age18to64'] + X['age65&over']
        X['pct_civadults_below_povertylvl'] = (X.nonvet_povertylvl_18to64 + X.vet_povertylvl_18to64 +X['vet_povertylvl_65&over'] + X['nonvet_povertylvl_65&over'])/ X['civilian_adults_total']                      
        
        #pct of age range NOT total pop
        X['disabilities_18to64'] = (X.nonvet_povertylvl_disability_18to64+ X.nonvet_abovepovertylvl_disability_18to64+ X.vet_povertylvl_disability_18to64 + X.vet_abovepovertylvl_disability_18to64)/X.age18to64
        X['disabilities_65&over'] = (X['nonvet_povertylvl_disability_65&over']+X['nonvet_abovepovertylvl_disability_65&over']+X['vet_povertylvl_disability_65&over']+X['vet_abovepovertylvl_disability_65&over'])/X['age65&over']
        X = X.drop(['age18to64', 'age65&over', 'nonvet_povertylvl_disability_65&over', 'nonvet_abovepovertylvl_disability_65&over','vet_povertylvl_disability_65&over','vet_abovepovertylvl_disability_65&over', 'nonvet_povertylvl_disability_18to64', 'nonvet_abovepovertylvl_disability_18to64','vet_povertylvl_disability_18to64','vet_abovepovertylvl_disability_18to64'], axis=1)
        
        #pct of owner occupied homes with 1 or more people per room
        X['OneOrMore_PersonPerRoom_Owner'] = (X['owner_occupied_1.01to1.5_PeoplePerRoom'] +X['owner_occupied_1.51to2_PeoplePerRoom'] + X['owner_occupied_2.01orMore_PeoplePerRoom'])/X.owner_occupied_units
        #pct of renter...
        X['OneOrMore_PersonPerRoom_Renter'] = (X['renter_occupied_1.01to1.5_PeoplePerRoom']+X['renter_occupied_1.5to2_PeoplePerRoom']+X['renter_occupied_2.01orMore_PeoplePerRoom'])/X.renter_occupied_units
        
        #pct males in total pop
        X['pct_males'] = X.Males/ X.population_total
        
        #have not received high school diploma or GED and are 25 or older
        grades = X.loc[:, X.columns.str.contains(' grade')] #.sum(axis=1)   # 'No schooling completed'  'Nursery school'  'Kindergarten'
        non_grades = X[['No schooling completed','Nursery school','Kindergarten']]
        X['pct_25andOver_NoHighSchoolDiploma'] = pd.concat([grades,non_grades], axis=1).sum(axis=1)/X['age25&over']
        X = X.drop((list(grades.columns)+list(non_grades.columns)+['Some college less than 1 year','Regular high school diploma','GED or alternative credential','age25&over']), axis=1)
        
        #proportion of pop with at least one child under 18
        X['have_child_under18'] = X['have_child_under18']/X.population_total
        
        if self.matrix:
            return X.values
        else: 
            return X            

In [9]:
att_trans = TransformationsPipeline()
processed = att_trans.transform(census_table)
processed.head()

,census_block_group,population_total,have_child_under18,renter_occupied_units,total_occupied_units,owner_occupied_units,nonvet_povertylvl_18to64,vet_povertylvl_18to64,nonvet_povertylvl_65&over,vet_povertylvl_65&over,...,pct_65over,no_health_under65,civilian_adults_total,pct_civadults_below_povertylvl,disabilities_18to64,disabilities_65&over,OneOrMore_PersonPerRoom_Owner,OneOrMore_PersonPerRoom_Renter,pct_males,pct_25andOver_NoHighSchoolDiploma
0,110010007014,1609,0.162213,259,705,446,229,0,0,0,...,0.111249,0.000000,1348,0.169881,0.071856,0.106145,0.0,0.050193,0.584214,0.015111
1,240338017043,1144,0.033217,181,495,314,121,0,31,0,...,0.221154,0.053322,1029,0.147716,0.137887,0.339921,0.0,0.088398,0.496503,0.093047
2,240338019081,1623,0.234750,580,618,38,62,8,12,0,...,0.091805,0.048059,1186,0.069140,0.111861,0.275168,0.0,0.000000,0.402957,0.140919
3,240317054002,1734,0.265859,34,582,548,13,4,0,0,...,0.197809,0.004614,1260,0.013492,0.042530,0.265306,0.0,0.000000,0.508651,0.000000
4,110010078091,1981,0.183746,332,804,472,242,0,30,0,...,0.102978,0.037355,1523,0.178595,0.114481,0.328431,0.0,0.018072,0.493185,0.148551


In [39]:
processed.to_csv('/project/sdscap-shakeri/location/Census_Data/census_table_transformed.csv', index=False)

In [10]:
geom = gpd.read_file('Census_Data/cbg.geojson')

In [11]:
user_list = pd.read_excel('DCBalt_user_list.xlsx')
user_list = user_list[user_list.batch_id.str.contains('DC')]
user_list = gpd.GeoDataFrame(user_list, geometry = gpd.points_from_xy(user_list.home_long,user_list.home_lat))

user_list = gpd.sjoin(user_list, geom, op='within')

/apps/software/standard/compiler/gcc/9.2.0/jupyter_conda/2020.11-py3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3338: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):
<ipython-input-11-7824cc55fd30>:5: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  user_list = gpd.sjoin(user_list, geom, op='within')


In [12]:
user_list.rename({'CensusBlockGroup':'census_block_group'},axis=1,inplace=True)
user_list.census_block_group = user_list.census_block_group.astype(int)
processed.census_block_group = processed.census_block_group.astype(int)
user_list.drop(['MTFCC','TractCode','CountyFIPS','StateFIPS','index_right','geometry','BlockGroup'], axis=1, inplace=True)

user_list_census = user_list.merge(processed, on='census_block_group')

In [13]:
user_list_census.head()

,advertiser_id,home_long,home_lat,batch_id,census_block_group,State,County,population_total,have_child_under18,renter_occupied_units,...,pct_65over,no_health_under65,civilian_adults_total,pct_civadults_below_povertylvl,disabilities_18to64,disabilities_65&over,OneOrMore_PersonPerRoom_Owner,OneOrMore_PersonPerRoom_Renter,pct_males,pct_25andOver_NoHighSchoolDiploma
0,0004C883-EE59-4B5E-8906-328330CFE93E,-77.073590,38.92840,DC0,110010007014,DC,District of Columbia,1609,0.162213,259,...,0.111249,0.0,1348,0.169881,0.071856,0.106145,0.0,0.050193,0.584214,0.015111
1,2F114A6E-3A76-4CA4-A96E-DEC601626A39,-77.073240,38.92749,DC3000,110010007014,DC,District of Columbia,1609,0.162213,259,...,0.111249,0.0,1348,0.169881,0.071856,0.106145,0.0,0.050193,0.584214,0.015111
2,42963756-218C-43BB-9A36-25FCC56DACF0,-77.076320,38.92984,DC4000,110010007014,DC,District of Columbia,1609,0.162213,259,...,0.111249,0.0,1348,0.169881,0.071856,0.106145,0.0,0.050193,0.584214,0.015111
3,A44E6290-7C6A-4E5D-8B25-781F1E79A6C1,-77.073595,38.92728,DC10000,110010007014,DC,District of Columbia,1609,0.162213,259,...,0.111249,0.0,1348,0.169881,0.071856,0.106145,0.0,0.050193,0.584214,0.015111
4,C3E07D07-0EF0-4733-B9B8-59C393A63D85,-77.073505,38.92876,DC12000,110010007014,DC,District of Columbia,1609,0.162213,259,...,0.111249,0.0,1348,0.169881,0.071856,0.106145,0.0,0.050193,0.584214,0.015111


In [14]:
user_list_census.to_csv('/project/sdscap-shakeri/location/user_list_census.csv')